# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
cv_seeds = [13, 29, 42, 55, 73]

## Load data

In [5]:
data_name = 'adult'

In [6]:
x, y, a = load_data(data_name)
raw_data = (x, y, a)

In [7]:
xdim = x.shape[1]
ydim = y.shape[1]
adim = a.shape[1]
zdim = 8

## Result file

In [8]:
header = "model_name", "cv_seed","fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [9]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.07506154477596283 | 0.3479534387588501 | 0.613844096660614 | 0.9193267822265625 | 0.7203947368421053 | 0.6301872469635628 | 0.02634741902834008
> 2 | 0.06706084311008453 | 0.33048874139785767 | 0.6465309858322144 | 0.9193267822265625 | 0.8338815789473685 | 0.6619433198380567 | 0.02634741902834008
> 3 | 0.06550358235836029 | 0.32480233907699585 | 0.6414579153060913 | 0.9193267822265625 | 0.8362221659919028 | 0.65625 | 0.02634741902834008
> 4 | 0.06518200039863586 | 0.32349780201911926 | 0.6397590637207031 | 0.9193267822265625 | 0.8393218623481782 | 0.657230516194332 | 0.02634741902834008
> 5 | 0.06530153006315231 | 0.32359111309051514 | 0.638432502746582 | 0.9193267822265625 | 0.8406819331983806 | 0.6577682186234818 | 0.02634741902834008
> 6 | 0.06536996364593506 | 0.32361042499542236 | 0.6375027298927307 | 0.9193267822265625 | 0.8416940789473685 | 0.6581161437246964 | 0.02634741902834008
> 7

### LAFTR for Eq Odds

In [10]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.095814548432827 | 0.36052340269088745 | 0.6479669809341431 | 0.9193267822265625 | 0.7141953441295547 | 0.625 | 0.02634741902834008
> 2 | -0.10186993330717087 | 0.33851492404937744 | 0.6441246867179871 | 0.9193267822265625 | 0.8287575910931174 | 0.661880060728745 | 0.02634741902834008
> 3 | -0.1032622829079628 | 0.33298298716545105 | 0.6427698135375977 | 0.9193267822265625 | 0.8341662449392713 | 0.6649797570850202 | 0.02634741902834008
> 4 | -0.10386868566274643 | 0.33013954758644104 | 0.6417455673217773 | 0.9193267822265625 | 0.8367914979757085 | 0.6648532388663968 | 0.02634741902834008
> 5 | -0.1040317639708519 | 0.32915735244750977 | 0.6412526369094849 | 0.9193267822265625 | 0.839132085020243 | 0.665296052631579 | 0.02634741902834008
> 6 | -0.10383305698633194 | 0.32954227924346924 | 0.6410415172576904 | 0.9193267822265625 | 0.841567560728745 | 0.6656756072874493 | 0.02634741902834008
> 7

### LAFTR for Eq Opp

In [11]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(
            xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.06292326748371124 | 0.3644852638244629 | 0.6521345376968384 | 0.9193267822265625 | 0.7194458502024291 | 0.6035551619433198 | 0.02634741902834008
> 2 | -0.06920120120048523 | 0.34251391887664795 | 0.6588622331619263 | 0.9193267822265625 | 0.827397520242915 | 0.6607730263157895 | 0.02634741902834008
> 3 | -0.07002678513526917 | 0.3380296230316162 | 0.6581521034240723 | 0.9193267822265625 | 0.8287259615384616 | 0.665043016194332 | 0.02634741902834008
> 4 | -0.06997490674257278 | 0.3378604054450989 | 0.6577456593513489 | 0.9193267822265625 | 0.832268471659919 | 0.6646318319838057 | 0.02634741902834008
> 5 | -0.0700741708278656 | 0.33687150478363037 | 0.6572105884552002 | 0.9193267822265625 | 0.8353049089068826 | 0.6646002024291497 | 0.02634741902834008
> 6 | -0.07022169232368469 | 0.33583030104637146 | 0.6568437814712524 | 0.9193267822265625 | 0.8379301619433198 | 0.6647267206477733 | 0.0263474

## Saving into DF then CSV

In [12]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,cv_seed,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,13,0.2,0.848786,0.823688,0.922764,0.919181,0.836048,0.884230,0.882582,3759.0,110.0,232.0,279.0,5647.0,642.0,1058.0,1777.0
1,LAFTR4DP,13,0.5,0.849082,0.844694,0.958788,0.976855,0.846882,0.900606,0.908498,3736.0,133.0,211.0,300.0,5700.0,589.0,1105.0,1730.0
2,LAFTR4DP,13,0.7,0.849378,0.859279,0.967224,0.983948,0.854300,0.904479,0.911724,3724.0,145.0,198.0,313.0,5742.0,547.0,1144.0,1691.0
3,LAFTR4DP,13,1.0,0.848415,0.869088,0.960648,0.964514,0.858627,0.901050,0.902747,3720.0,149.0,196.0,315.0,5775.0,514.0,1188.0,1647.0
4,LAFTR4DP,29,0.2,0.848786,0.819025,0.911814,0.899777,0.833639,0.879172,0.873538,3786.0,92.0,243.0,263.0,5651.0,627.0,1080.0,1762.0
5,LAFTR4DP,29,0.5,0.849230,0.831824,0.937349,0.942739,0.840437,0.891116,0.893545,3770.0,108.0,225.0,281.0,5676.0,602.0,1101.0,1741.0
6,LAFTR4DP,29,0.7,0.850341,0.852540,0.967794,0.988270,0.851439,0.905274,0.914132,3758.0,120.0,213.0,293.0,5753.0,525.0,1163.0,1679.0
7,LAFTR4DP,29,1.0,0.847971,0.869826,0.965895,0.972966,0.858759,0.903100,0.906179,3735.0,143.0,203.0,303.0,5788.0,490.0,1217.0,1625.0
8,LAFTR4DP,42,0.2,0.852488,0.792282,0.908622,0.912026,0.821283,0.879660,0.881253,3806.0,119.0,200.0,299.0,5441.0,778.0,895.0,1966.0
9,LAFTR4DP,42,0.5,0.850563,0.813449,0.946545,0.974103,0.831592,0.895991,0.908151,3769.0,156.0,176.0,323.0,5468.0,751.0,935.0,1926.0


In [13]:
result_df.to_csv(f'{data_name}-result/laftr-{epochs}.csv')